In [ ]:
# Tool importing section
import os
from glob import glob  # File manipulation
import matplotlib.pyplot as plt
# Use geopandas for vector data and rasterio for raster data
import geopandas as gpd
# Plotting extent is used to plot raster & vector data together
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import pathlib
from pathlib import Path
import fiona
import pandas as pd

In [ ]:
os.chdir(os.path.join('P:\\Personal Files\\Education\\FRCC\\NSF_Internship\\NSF_Project_Files\\Data'))
allsoil_nd=os.path.join("Soil_Original_Data","soilmu_a_nd085.shp")
allsoil_sd=os.path.join("Soil_Original_Data","soilmu_a_sd031.shp")
table_org=os.path.join("Preped_CSVs","muaggatt.csv")
concat_soil=os.path.join("Soil_Results","merged_soil.shp")
soil_join_file=os.path.join("Soil_Results","hydro_group_join_soil.shp")
final_soil = os.path.join("Final_Results","Input_Layers","Soil_Hydro_Group.shp")
soil_fig = os.path.join("Figures","Soil_Hydro_Group.png")
diss_field='Hydrologic_Group_Dominant_Conditions'
join_head=['AREASYMBOL','SPATIALVER','Mapunit_Symbol','Mapunit_Name','MUKEY','geometry',diss_field]

In [ ]:
projection = "epsg:26914"

In [ ]:
#Get Reservation boundary
res_path = os.path.join("Reservation_Boundary_Layer","BIA_National_LAR.shp")
res_boundary = gpd.read_file(res_path)
res_aoi = res_boundary[res_boundary["LARName"] == "Standing Rock LAR"]
res_projected = res_aoi.to_crs(projection)

In [ ]:
soils=gpd.read_file(allsoil_sd)
soiln=gpd.read_file(allsoil_nd)
soils_proj = soils.to_crs(projection)
soiln_proj = soiln.to_crs(projection)

In [ ]:
all_soil=gpd.GeoDataFrame(pd.concat([soils_proj,soiln_proj]))
all_soil.to_file(concat_soil)
all_soil.head()

In [ ]:
table_r = pd.read_csv(table_org)
all_soil['MUKEY']=all_soil.MUKEY.astype(str)
table_r['Mapunit_Key']=table_r.Mapunit_Key.astype(str)
table_r.rename(columns={'Mapunit_Key':'MUKEY'},inplace=True)
table_r.head()

In [ ]:
f, ax1 = plt.subplots()
all_soil.plot(ax=ax1)
res_projected.plot(color='None',
            Edgecolor='black',ax=ax1)
ax1.set(title="Soil Layer")
plt.show()

In [ ]:
soil_join = all_soil.merge(table_r, on='MUKEY', how='outer')
soil_join.head()

In [ ]:
column_names = list(soil_join)
column_names

In [ ]:
soil_join_reduced = soil_join[join_head]
column_names2 = list(soil_join_reduced)
print(column_names2)
print(soil_join_reduced.head())
soil_join_reduced.to_file(soil_join_file)

In [ ]:
soil_reduced = gpd.read_file(soil_join_file)
diss_field_cut=diss_field[:10]
soil_dissolve_value = soil_reduced[[diss_field_cut,'geometry']]
soil_dissolve = soil_dissolve_value.dissolve(by=diss_field_cut)
soil_dissolve

In [ ]:
soil_clip = gpd.clip(soil_dissolve,res_projected)

In [ ]:
f, ax1 = plt.subplots()
soil_clip.plot(legend=True,
               ax=ax1)
res_projected.plot(color='None',
            Edgecolor='black',ax=ax1)
ax1.set(title="Soil by Hydrologic Group")
plt.show()
plt.draw()
f.savefig(soil_fig, dpi=400)

In [ ]:
soil_clip.to_file(final_soil)

In [ ]:
#End of Soil Prep